# Enable Agent Tutorial Part 4: RAG 챗봇 구현

## 개요

Part 3에서 구축한 지식 베이스를 활용하여 대화형 RAG 챗봇을 구현한다.

## 학습 내용

- RAG 챗봇 구현
- 대화형 예측 인터페이스
- 자동 분석 보고서 생성
- 통합 데모

---
## 1. 라이브러리 임포트

In [1]:
import os
import json
import yaml
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from typing import List, Dict, Any, Tuple

import torch
import torch.nn as nn

from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
print("라이브러리 임포트 완료")

라이브러리 임포트 완료


---
## 2. 이전 Part에서 정의한 클래스 재사용

In [2]:
class SpaceshipClassifier(nn.Module):
    """Spaceship Titanic 이진 분류 신경망"""
    
    def __init__(self, input_dim=10, hidden_dims=[64, 32, 16], dropout_rate=0.3):
        super(SpaceshipClassifier, self).__init__()
        
        layers = []
        prev_dim = input_dim
        
        for i, hidden_dim in enumerate(hidden_dims):
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.ReLU())
            if i < len(hidden_dims) - 1:
                layers.append(nn.Dropout(dropout_rate))
            prev_dim = hidden_dim
        
        layers.append(nn.Linear(prev_dim, 1))
        layers.append(nn.Sigmoid())
        
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x).squeeze()

print("SpaceshipClassifier 정의 완료")

SpaceshipClassifier 정의 완료


In [3]:
class SpaceshipEnableAgent:
    """Spaceship Titanic 예측 모델을 Enable Agent로 래핑한 클래스"""
    
    def __init__(self, skill_path: str):
        with open(skill_path, 'r', encoding='utf-8') as f:
            self.skill = yaml.safe_load(f)
        
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        model_path = self.skill['model_info']['model_path']
        checkpoint = torch.load(model_path, map_location=self.device, weights_only=True)
        
        self.model = SpaceshipClassifier(
            input_dim=checkpoint['input_dim'],
            hidden_dims=checkpoint['hidden_dims'],
            dropout_rate=checkpoint['dropout_rate']
        ).to(self.device)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.model.eval()
        
        scaler_path = self.skill['model_info']['scaler_path']
        self.scaler = joblib.load(scaler_path)
        
        metadata_path = self.skill['model_info']['metadata_path']
        with open(metadata_path, 'r', encoding='utf-8') as f:
            self.metadata = json.load(f)
        
        self.encoding_maps = self.skill['encoding_maps']
        self.client = OpenAI()
        
        print(f"Enable Agent 초기화 완료: {self.skill['agent_name']}")
    
    def get_capability_description(self) -> str:
        return f"""
Agent 이름: {self.skill['agent_name']}
버전: {self.skill['version']}
설명: {self.skill['description']}

입력 파라미터:
- HomePlanet: 출발 행성 (0: Earth, 1: Europa, 2: Mars, 3: Unknown)
- CryoSleep: 냉동 수면 여부 (0: No, 1: Yes)
- Destination: 목적지 (0: TRAPPIST-1e, 1: PSO J318.5-22, 2: 55 Cancri e)
- Age: 나이
- VIP: VIP 서비스 여부 (0: No, 1: Yes)
- RoomService, FoodCourt, ShoppingMall, Spa, VRDeck: 각 서비스 지출액

모델 정확도: {self.metadata['test_accuracy']:.4f}
AUC-ROC: {self.metadata['test_auc_roc']:.4f}
""".strip()
    
    def _decode_categorical(self, input_data: Dict[str, Any]) -> Dict[str, str]:
        decoded = {}
        for field, value in input_data.items():
            if field in self.encoding_maps:
                decoded[field] = self.encoding_maps[field].get(value, str(value))
            else:
                decoded[field] = value
        return decoded
    
    def predict(self, input_data: Dict[str, Any]) -> Dict[str, Any]:
        feature_names = self.metadata['feature_names']
        input_array = np.array([[input_data[f] for f in feature_names]])
        
        input_scaled = self.scaler.transform(input_array)
        input_tensor = torch.FloatTensor(input_scaled).to(self.device)
        
        with torch.no_grad():
            probability = self.model(input_tensor).item()
        
        predicted_class = 'Transported' if probability > 0.5 else 'Not Transported'
        confidence = probability if probability > 0.5 else (1 - probability)
        
        decoded_features = self._decode_categorical(input_data)
        
        return {
            "predicted_class": predicted_class,
            "probability": float(probability),
            "confidence": float(confidence),
            "input_features": input_data,
            "decoded_features": decoded_features,
            "timestamp": datetime.now().isoformat()
        }
    
    def generate_tool_definition(self) -> Dict[str, Any]:
        return {
            "type": "function",
            "function": {
                "name": "predict_spaceship_transport",
                "description": self.skill['description'],
                "parameters": {
                    "type": "object",
                    "properties": {
                        "HomePlanet": {"type": "integer", "description": "출발 행성 (0: Earth, 1: Europa, 2: Mars)"},
                        "CryoSleep": {"type": "integer", "description": "냉동 수면 여부 (0: No, 1: Yes)"},
                        "Destination": {"type": "integer", "description": "목적지 (0: TRAPPIST-1e, 1: PSO J318.5-22, 2: 55 Cancri e)"},
                        "Age": {"type": "number", "description": "나이"},
                        "VIP": {"type": "integer", "description": "VIP 여부 (0: No, 1: Yes)"},
                        "RoomService": {"type": "number", "description": "룸서비스 지출액"},
                        "FoodCourt": {"type": "number", "description": "푸드코트 지출액"},
                        "ShoppingMall": {"type": "number", "description": "쇼핑몰 지출액"},
                        "Spa": {"type": "number", "description": "스파 지출액"},
                        "VRDeck": {"type": "number", "description": "VR 데크 지출액"}
                    },
                    "required": ["HomePlanet", "CryoSleep", "Destination", "Age", "VIP",
                                 "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]
                }
            }
        }

print("SpaceshipEnableAgent 정의 완료")

SpaceshipEnableAgent 정의 완료


In [4]:
class PredictionContextBuilder:
    """예측 결과를 컨텍스트로 저장하고 관리하는 클래스"""
    
    def __init__(self, context_dir: str = 'context_store'):
        self.context_dir = Path(context_dir)
        self.context_dir.mkdir(exist_ok=True)
        
        self.log_file = self.context_dir / 'prediction_logs.json'
        self.summary_file = self.context_dir / 'prediction_summary.json'
        self.knowledge_base_file = self.context_dir / 'knowledge_base.txt'
        
        self.logs = self._load_logs()
        
        print(f"Context Builder 초기화 완료")
        print(f"저장된 예측 로그: {len(self.logs)}개")
    
    def _load_logs(self) -> List[Dict[str, Any]]:
        if self.log_file.exists():
            with open(self.log_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        return []
    
    def add_prediction(self, prediction_result: Dict[str, Any]):
        self.logs.append(prediction_result)
        
        with open(self.log_file, 'w', encoding='utf-8') as f:
            json.dump(self.logs, f, indent=2, ensure_ascii=False)
        
        self._update_summary()
        self._update_knowledge_base()
    
    def _update_summary(self):
        if not self.logs:
            return
        
        total_predictions = len(self.logs)
        class_counts = {'Transported': 0, 'Not Transported': 0}
        probability_sum = 0
        confidence_sum = 0
        
        for log in self.logs:
            class_counts[log['predicted_class']] = class_counts.get(log['predicted_class'], 0) + 1
            probability_sum += log['probability']
            confidence_sum += log['confidence']
        
        summary = {
            "total_predictions": total_predictions,
            "class_distribution": class_counts,
            "average_probability": probability_sum / total_predictions,
            "average_confidence": confidence_sum / total_predictions,
            "last_updated": datetime.now().isoformat()
        }
        
        with open(self.summary_file, 'w', encoding='utf-8') as f:
            json.dump(summary, f, indent=2, ensure_ascii=False)
    
    def _update_knowledge_base(self):
        if not self.logs:
            return
        
        with open(self.summary_file, 'r', encoding='utf-8') as f:
            summary = json.load(f)
        
        knowledge_text = f"""# Spaceship Titanic 예측 지식 베이스

## 전체 통계
- 총 예측 횟수: {summary['total_predictions']}회
- 평균 이동 확률: {summary['average_probability']:.2%}
- 평균 신뢰도: {summary['average_confidence']:.2%}

## 예측 결과 분포
"""
        for cls, count in summary['class_distribution'].items():
            pct = (count / summary['total_predictions']) * 100
            knowledge_text += f"- {cls}: {count}회 ({pct:.1f}%)\n"
        
        knowledge_text += "\n## 최근 예측 (최근 5개)\n\n"
        for i, log in enumerate(reversed(self.logs[-5:]), 1):
            decoded = log.get('decoded_features', {})
            knowledge_text += f"""### 예측 #{len(self.logs) - i + 1}
- 결과: {log['predicted_class']} (확률: {log['probability']:.2%})
- 출발: {decoded.get('HomePlanet', 'N/A')}, 냉동수면: {decoded.get('CryoSleep', 'N/A')}

"""
        
        with open(self.knowledge_base_file, 'w', encoding='utf-8') as f:
            f.write(knowledge_text.strip())
    
    def get_knowledge_base_content(self) -> str:
        if self.knowledge_base_file.exists():
            with open(self.knowledge_base_file, 'r', encoding='utf-8') as f:
                return f.read()
        return ""
    
    def get_summary(self) -> Dict[str, Any]:
        if self.summary_file.exists():
            with open(self.summary_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        return {}

print("PredictionContextBuilder 정의 완료")

PredictionContextBuilder 정의 완료


---
## 3. SpaceshipRAGChatbot 클래스 구현

### 핵심 기능

- **지식 베이스 통합**: 과거 예측 결과를 컨텍스트로 활용
- **Function Calling**: 실시간 예측 수행
- **자동 컨텍스트 업데이트**: 새로운 예측을 지식 베이스에 추가
- **보고서 생성**: 전체 예측 결과 분석

In [5]:
class SpaceshipRAGChatbot:
    """Spaceship Titanic 예측 지식을 활용하는 RAG 챗봇"""
    
    def __init__(self, agent: SpaceshipEnableAgent, context_builder: PredictionContextBuilder):
        self.agent = agent
        self.context_builder = context_builder
        self.client = OpenAI()
        self.conversation_history = []
        
        print("RAG 챗봇 초기화 완료")
    
    def _get_system_prompt(self) -> str:
        knowledge_base = self.context_builder.get_knowledge_base_content()
        agent_capability = self.agent.get_capability_description()
        
        return f"""
당신은 Spaceship Titanic 승객의 다른 차원 이동 여부를 예측하는 AI 어시스턴트다.
다음과 같은 역할을 수행한다:

1. 사용자의 승객 정보를 받아 이동 여부를 예측한다
2. 과거 예측 기록을 바탕으로 통계와 인사이트를 제공한다
3. Spaceship Titanic의 배경과 특성에 대해 설명한다
4. 예측 결과에 대한 상세한 분석과 해석을 제공한다

## 사용 가능한 도구

{agent_capability}

## 과거 예측 지식 베이스

{knowledge_base}

## 응답 가이드라인

- 친절하고 전문적인 톤을 유지한다
- 기술 용어를 사용할 때는 쉬운 설명을 덧붙인다
- 과거 예측 데이터를 활용하여 맥락있는 답변을 제공한다
- 예측 요청 시 predict_spaceship_transport 함수를 호출한다
- 문장의 끝은 ~다로 끝낸다
""".strip()
    
    def chat(self, user_message: str) -> str:
        self.conversation_history.append({"role": "user", "content": user_message})
        
        messages = [{"role": "system", "content": self._get_system_prompt()}] + self.conversation_history
        tools = [self.agent.generate_tool_definition()]
        
        response = self.client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            tools=tools,
            tool_choice="auto",
            temperature=0.7
        )
        
        response_message = response.choices[0].message
        
        if response_message.tool_calls:
            for tool_call in response_message.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                
                if function_name == "predict_spaceship_transport":
                    prediction_result = self.agent.predict(function_args)
                    self.context_builder.add_prediction(prediction_result)
                    
                    self.conversation_history.append({
                        "role": "assistant",
                        "content": None,
                        "tool_calls": [tool_call]
                    })
                    
                    self.conversation_history.append({
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "name": function_name,
                        "content": json.dumps(prediction_result, ensure_ascii=False)
                    })
            
            messages = [{"role": "system", "content": self._get_system_prompt()}] + self.conversation_history
            
            final_response = self.client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0.7
            )
            
            assistant_message = final_response.choices[0].message.content
        else:
            assistant_message = response_message.content
        
        self.conversation_history.append({"role": "assistant", "content": assistant_message})
        
        return assistant_message
    
    def generate_report(self) -> str:
        summary = self.context_builder.get_summary()
        knowledge_base = self.context_builder.get_knowledge_base_content()
        
        messages = [
            {
                "role": "system",
                "content": "당신은 데이터 분석 보고서를 작성하는 전문가다. 주어진 예측 데이터를 분석하여 경영진이 이해하기 쉬운 보고서를 작성한다. 문장의 끝은 ~다로 끝낸다."
            },
            {
                "role": "user",
                "content": f"""
다음 Spaceship Titanic 예측 데이터를 바탕으로 상세한 분석 보고서를 작성해달라:

## 요약 통계
{json.dumps(summary, indent=2, ensure_ascii=False)}

## 상세 예측 기록
{knowledge_base}

보고서에 다음 내용을 포함해달라:
1. 전체 예측 성능 요약
2. 이동/비이동 패턴 분석
3. 주요 영향 요인 분석
4. 인사이트 및 권장사항
5. 향후 개선 방향
"""
            }
        ]
        
        response = self.client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            temperature=0.7,
            max_tokens=2000
        )
        
        return response.choices[0].message.content
    
    def reset_conversation(self):
        self.conversation_history = []
        print("대화 히스토리 초기화 완료")

print("SpaceshipRAGChatbot 클래스 정의 완료")

SpaceshipRAGChatbot 클래스 정의 완료


---
## 4. RAG 챗봇 초기화

In [6]:
agent = SpaceshipEnableAgent('skills/spaceship_agent_skill.yaml')
context_builder = PredictionContextBuilder()
chatbot = SpaceshipRAGChatbot(agent, context_builder)

print("\n모든 컴포넌트 초기화 완료")

Enable Agent 초기화 완료: SpaceshipTransportPredictor
Context Builder 초기화 완료
저장된 예측 로그: 10개
RAG 챗봇 초기화 완료

모든 컴포넌트 초기화 완료


---
## 5. 대화형 데모

RAG 챗봇과 대화하며 승객의 다른 차원 이동 여부를 예측한다.

In [7]:
def chat_demo(user_input: str) -> str:
    print("\n" + "="*70)
    print(f"사용자: {user_input}")
    print("-"*70)
    
    response = chatbot.chat(user_input)
    
    print(f"챗봇: {response}")
    print("="*70)
    
    return response

In [8]:
chat_demo("안녕하세요! Spaceship Titanic에 대해 알려주세요.")


사용자: 안녕하세요! Spaceship Titanic에 대해 알려주세요.
----------------------------------------------------------------------
챗봇: 안녕하세요! Spaceship Titanic은 미래의 우주 탐사에서 중요한 역할을 하는 거대한 우주선이다. 이 우주선은 다양한 출발 행성에서 온 승객들을 여러 목적지로 이동시키는 임무를 수행한다. 승객들은 다양한 이유로 여행을 떠나며, 이 중 일부는 다른 차원으로 이동할 수 있는 특별한 기회를 얻는다. 

Spaceship Titanic의 독특한 점은 차원 이동 기술을 사용하는 것이다. 승객들은 여행 중 냉동 수면(CryoSleep)을 선택할 수 있으며, 이 상태에서 차원 이동의 가능성이 높아진다. 또한, 우주선 내에서는 다양한 서비스가 제공되며, 승객들은 룸서비스, 푸드코트, 쇼핑몰, 스파, VR 데크 등에서 지출을 할 수 있다.

Spaceship Titanic의 목적지는 대표적으로 TRAPPIST-1e, PSO J318.5-22, 55 Cancri e가 있으며, 각 목적지는 독특한 환경과 탐험 기회를 제공한다. 승객의 출발 행성과 목적지, 여행 중의 활동 등에 따라 차원 이동 여부가 결정된다.


'안녕하세요! Spaceship Titanic은 미래의 우주 탐사에서 중요한 역할을 하는 거대한 우주선이다. 이 우주선은 다양한 출발 행성에서 온 승객들을 여러 목적지로 이동시키는 임무를 수행한다. 승객들은 다양한 이유로 여행을 떠나며, 이 중 일부는 다른 차원으로 이동할 수 있는 특별한 기회를 얻는다. \n\nSpaceship Titanic의 독특한 점은 차원 이동 기술을 사용하는 것이다. 승객들은 여행 중 냉동 수면(CryoSleep)을 선택할 수 있으며, 이 상태에서 차원 이동의 가능성이 높아진다. 또한, 우주선 내에서는 다양한 서비스가 제공되며, 승객들은 룸서비스, 푸드코트, 쇼핑몰, 스파, VR 데크 등에서 지출을 할 수 있다.\n\nSpaceship Titanic의 목적지는 대표적으로 TRAPPIST-1e, PSO J318.5-22, 55 Cancri e가 있으며, 각 목적지는 독특한 환경과 탐험 기회를 제공한다. 승객의 출발 행성과 목적지, 여행 중의 활동 등에 따라 차원 이동 여부가 결정된다.'

In [9]:
chat_demo("""
지구에서 출발한 25세 승객이 있어요.
냉동 수면 중이고, 목적지는 55 Cancri e입니다.
VIP는 아니고, 모든 서비스 지출이 0이에요.
이 승객이 다른 차원으로 이동될까요?
""")


사용자: 
지구에서 출발한 25세 승객이 있어요.
냉동 수면 중이고, 목적지는 55 Cancri e입니다.
VIP는 아니고, 모든 서비스 지출이 0이에요.
이 승객이 다른 차원으로 이동될까요?

----------------------------------------------------------------------
챗봇: 지구에서 출발한 25세 승객은 냉동 수면 중이며, 목적지는 55 Cancri e이다. VIP 서비스는 이용하지 않았고, 모든 서비스 지출이 0이다. 이 승객은 다른 차원으로 이동될 확률이 약 66.66%로 예측되었다. 이는 과거 예측 기록에서 냉동 수면 상태인 승객들이 다른 차원으로 이동될 가능성이 높았던 패턴과 일치한다.

이번 예측은 "Transported", 즉 다른 차원으로 이동될 것으로 예측되었으며, 확률이 66.66%로 비교적 높은 신뢰도를 보인다. 같은 조건에서 과거 예측에서도 냉동 수면 상태의 승객들이 이동될 가능성이 높았던 점을 고려하면, 이 승객도 다른 차원으로 이동될 가능성이 높다.


'지구에서 출발한 25세 승객은 냉동 수면 중이며, 목적지는 55 Cancri e이다. VIP 서비스는 이용하지 않았고, 모든 서비스 지출이 0이다. 이 승객은 다른 차원으로 이동될 확률이 약 66.66%로 예측되었다. 이는 과거 예측 기록에서 냉동 수면 상태인 승객들이 다른 차원으로 이동될 가능성이 높았던 패턴과 일치한다.\n\n이번 예측은 "Transported", 즉 다른 차원으로 이동될 것으로 예측되었으며, 확률이 66.66%로 비교적 높은 신뢰도를 보인다. 같은 조건에서 과거 예측에서도 냉동 수면 상태의 승객들이 이동될 가능성이 높았던 점을 고려하면, 이 승객도 다른 차원으로 이동될 가능성이 높다.'

In [10]:
chat_demo("""
이번에는 유로파에서 온 45세 VIP 승객이에요.
냉동 수면은 하지 않고, TRAPPIST-1e로 가고 있어요.
룸서비스 2000, 푸드코트 3000, 쇼핑몰 1000, 스파 5000, VR 데크 1500을 썼어요.
예측해주세요.
""")


사용자: 
이번에는 유로파에서 온 45세 VIP 승객이에요.
냉동 수면은 하지 않고, TRAPPIST-1e로 가고 있어요.
룸서비스 2000, 푸드코트 3000, 쇼핑몰 1000, 스파 5000, VR 데크 1500을 썼어요.
예측해주세요.

----------------------------------------------------------------------
챗봇: 유로파에서 온 45세 VIP 승객은 냉동 수면을 하지 않고 TRAPPIST-1e로 가고 있다. 이 승객은 룸서비스, 푸드코트, 쇼핑몰, 스파, VR 데크에서 상당한 금액을 지출하였다. 이러한 조건에서 예측한 결과, 이 승객은 다른 차원으로 이동되지 않을 확률이 매우 높다. 정확히 말하면, 다른 차원으로 이동되지 않을 확률이 99.92%로 예측되었다.

냉동 수면 상태가 아니고, VIP로서 많은 금액을 지출한 경우에도 다른 차원으로 이동될 확률이 낮음을 보여준다. 이와 같은 패턴은 과거 예측 기록에서도 자주 관찰되었으며, 냉동 수면을 하지 않는 승객들이 이동될 확률이 낮았다는 점과 일치한다.


'유로파에서 온 45세 VIP 승객은 냉동 수면을 하지 않고 TRAPPIST-1e로 가고 있다. 이 승객은 룸서비스, 푸드코트, 쇼핑몰, 스파, VR 데크에서 상당한 금액을 지출하였다. 이러한 조건에서 예측한 결과, 이 승객은 다른 차원으로 이동되지 않을 확률이 매우 높다. 정확히 말하면, 다른 차원으로 이동되지 않을 확률이 99.92%로 예측되었다.\n\n냉동 수면 상태가 아니고, VIP로서 많은 금액을 지출한 경우에도 다른 차원으로 이동될 확률이 낮음을 보여준다. 이와 같은 패턴은 과거 예측 기록에서도 자주 관찰되었으며, 냉동 수면을 하지 않는 승객들이 이동될 확률이 낮았다는 점과 일치한다.'

In [ ]:
chat_demo("지금까지의 예측 통계를 알려주세요.")


사용자: 지금까지의 예측 통계를 알려주세요.
----------------------------------------------------------------------


In [ ]:
chat_demo("냉동 수면이 이동 확률에 어떤 영향을 미치나요?")

---
## 6. 분석 보고서 생성

In [ ]:
print("분석 보고서 생성 중...")
report = chatbot.generate_report()

print("\n" + "="*70)
print("Spaceship Titanic 예측 분석 보고서")
print("="*70)
print(report)

In [ ]:
report_path = 'context_store/analysis_report.md'
with open(report_path, 'w', encoding='utf-8') as f:
    f.write(report)

print(f"보고서 저장 완료: {report_path}")

---
## 7. 시스템 현황 요약

In [ ]:
summary = context_builder.get_summary()

print("="*70)
print("Enable Agent 생태계 현황")
print("="*70)
print()
print("[1] Enable Agent:")
print(f"   - Agent 이름: {agent.skill['agent_name']}")
print(f"   - 모델 타입: {agent.metadata['model_type']}")
print(f"   - 테스트 정확도: {agent.metadata['test_accuracy']:.2%}")
print(f"   - AUC-ROC: {agent.metadata['test_auc_roc']:.2%}")
print()

if summary:
    print("[2] 예측 통계:")
    print(f"   - 총 예측 횟수: {summary['total_predictions']}회")
    print(f"   - 평균 신뢰도: {summary['average_confidence']:.2%}")
    print(f"   - 결과별 분포:")
    for cls, count in summary['class_distribution'].items():
        pct = (count / summary['total_predictions']) * 100
        print(f"     * {cls}: {count}회 ({pct:.1f}%)")
    print()

print("[3] 챗봇 활동:")
print(f"   - 대화 수: {len(chatbot.conversation_history)}개")
print()
print("[4] 생성된 파일:")
print(f"   - 예측 로그: {context_builder.log_file}")
print(f"   - 통계 요약: {context_builder.summary_file}")
print(f"   - 지식 베이스: {context_builder.knowledge_base_file}")
print(f"   - 분석 보고서: context_store/analysis_report.md")
print()
print("="*70)